In [2]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from utils import exportsDataPath, workingDataPath, writeHypnoDfParquet

import pandas as pd
import datetime
import os
import pytz
import json
print(pd.__version__)
utc = pytz.utc
AZtz = pytz.timezone("US/Arizona")

ColumnNames = ["startDate", "endDate", "value"]

jsonFileName = "oura_sleep_2024-09-13T00-24-27.json"

with open(exportsDataPath + 'oura/' + jsonFileName) as f:
  jsonData = json.load(f)

1.1.2


In [3]:
def ouraSleepStageToStandard(ouraNum):
    if ouraNum == 4: #awake
        return 0
    if ouraNum == 2: #light sleep
        return 1
    if ouraNum == 1: #deep sleep
        return 2
    if ouraNum == 3: #REM sleep
        return 3
    

In [6]:
#testing parsing
print(jsonData['sleep'][0]['bedtime_start'])
print(jsonData['sleep'][0]['bedtime_end'])
print(jsonData['sleep'][0]['sleep_phase_5_min'])
print(len(jsonData['sleep'][0]['sleep_phase_5_min']))
print(pd.to_datetime(jsonData['sleep'][0]['bedtime_start']) + datetime.timedelta(minutes=len(jsonData['sleep'][0]['sleep_phase_5_min'])*5))


2020-10-15T01:03:59.000-07:00
2020-10-15T09:47:59.000-07:00
424221114322222121112211123322211111122222211223312224333333242221122222333333442222222223333344444444444
105
2020-10-15 09:48:59-07:00


In [7]:
# testing generating a record
listRecords = []
startTime = pd.to_datetime(jsonData['sleep'][0]['bedtime_start'])

hypnogram_5min = jsonData['sleep'][0]['sleep_phase_5_min']
toadd = []
toadd.append(startTime)

for i in range(1, len(hypnogram_5min)):
    if(hypnogram_5min[i-1] != hypnogram_5min[i]):#if the block has ended
        toadd.append(startTime + datetime.timedelta(minutes=i*5)) # add the end time
        toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[i-1])))
        listRecords.append(toadd)
        toadd = [startTime + datetime.timedelta(minutes=i*5)]

toadd.append(startTime + datetime.timedelta(minutes=len(hypnogram_5min)*5))
toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[-1])))
listRecords.append(toadd)

In [8]:
listRecords[-1]

[Timestamp('2020-10-15 08:53:59-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2020-10-15 09:48:59-0700', tz='pytz.FixedOffset(-420)'),
 0]

In [42]:
#parse the file
listRecords = []

for SleepJSON in jsonData['sleep']:
    # if sleep_phase_5_min key exists parse that
    if 'sleep_phase_5_min' in SleepJSON:
            toadd = []
            startTime = pd.to_datetime(SleepJSON['bedtime_start']).tz_convert("US/Arizona")
            toadd.append(startTime)

            hypnogram_5min = SleepJSON['sleep_phase_5_min']
            for i in range(1, len(hypnogram_5min)):
                if(hypnogram_5min[i-1] != hypnogram_5min[i]):#if the block has ended
                    toadd.append(startTime + datetime.timedelta(minutes=i*5)) # add the end time
                    toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[i-1]))) #add the value
                    listRecords.append(toadd)
                    #start time of the next block
                    toadd = [startTime + datetime.timedelta(minutes=i*5)]

            toadd.append(startTime + datetime.timedelta(minutes=len(hypnogram_5min)*5))
            toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[-1])))
            listRecords.append(toadd)

    # if it doesn't if total_sleep_duration exists and is == 0 then label the bedime start and end as awake time
    # elif 'total_sleep_duration' in SleepJSON and SleepJSON['total_sleep_duration'] == 0:
    #     startTime = pd.to_datetime(SleepJSON['bedtime_start']).tz_convert("US/Arizona")
    #     endTime = pd.to_datetime(SleepJSON['bedtime_end']).tz_convert("US/Arizona")
    #     listRecords.append([startTime, endTime, 0])


    # # if total_sleep_duration also doesn't exit label the start and end times as stage 1
    # else:
    #     startTime = pd.to_datetime(SleepJSON['bedtime_start']).tz_convert("US/Arizona")
    #     endTime = pd.to_datetime(SleepJSON['bedtime_end']).tz_convert("US/Arizona")
    #     listRecords.append([startTime, endTime, 1])



In [43]:
len(listRecords)

28206

In [44]:
set(jsonData.keys())

{'sleep'}

In [45]:
# for restfulJSON in jsonData['restful_periods']:
#     startTime = pd.to_datetime(restfulJSON['bedtime_start']).tz_convert("US/Arizona")
#     endTime = pd.to_datetime(restfulJSON['bedtime_end']).tz_convert("US/Arizona")
#     listRecords.append([startTime, endTime, 1]) #fill in naps with light sleep

In [46]:
len(listRecords)

28206

In [47]:
listRecords = sorted(listRecords, key=lambda x:x[0])

In [48]:
# add in the no data segments
filledRecords = []
# for every row except the last
for ri in range(len(listRecords)-1):
    filledRecords.append(listRecords[ri])
    # if there is a gap between this rows end date and the next rows start date
    if listRecords[ri][1] < listRecords[ri+1][0]:
        filledRecords.append([
            listRecords[ri][1],
            listRecords[ri+1][0],
            -1
        ])
print(len(filledRecords))

30608


In [49]:
ouraHypnoDf = pd.DataFrame(columns=ColumnNames, data=filledRecords)
ouraHypnoDf = ouraHypnoDf.set_index("startDate")
print(ouraHypnoDf.head(1))
print(ouraHypnoDf.tail(1))

                                            endDate  value
startDate                                                 
2020-10-15 01:03:59-07:00 2020-10-15 01:08:59-07:00      0
                                            endDate  value
startDate                                                 
2023-08-30 03:15:03-07:00 2023-08-30 03:30:03-07:00      0


In [50]:
ouraHypnoDf.dtypes

endDate    datetime64[ns, US/Arizona]
value                           int64
dtype: object

In [51]:
writeHypnoDfParquet('oura', ouraHypnoDf)